<a href="https://colab.research.google.com/github/Veldora00/Veldora00/blob/main/winratefinder_for1/2_rr_with_50percent_above_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas_ta as ta
import numpy as np
import pandas as pd

# Load historical data
data = yf.download('EURUSD=X', start='2000-01-01', end='2023-01-01')

# Calculate RSI and ATR using pandas-ta
data['RSI'] = ta.rsi(data['Close'], length=14)
data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)

# Set a list of RSI thresholds to test
rsi_levels = range(20, 70, 5)  # Test RSI from 20 to 65 for longs, and >70 for shorts

# Initialize a dictionary to store results for longs and shorts
results = {'longs': {}, 'shorts': {}}

# Loop through different RSI thresholds for longs
for rsi_threshold in rsi_levels:
    print(f"Testing RSI long threshold: {rsi_threshold}")  # Debug print to track progress

    # Define a buy condition based on the RSI threshold (longs)
    buy_condition = data['RSI'] < rsi_threshold

    # Track results for longs
    win_trades = []
    lose_trades = []

    # Simulate trades for each buy signal
    for i in range(1, len(data)):
        if buy_condition.iloc[i]:  # Long condition
            entry_price = data['Close'].iloc[i]
            stop_loss = entry_price - data['ATR'].iloc[i]  # Risk 1x ATR as stop loss
            take_profit = entry_price + 2 * (entry_price - stop_loss)  # 2x risk as take profit

            # Simulate the trade outcome
            for j in range(i + 1, len(data)):
                if data['Low'].iloc[j] <= stop_loss:
                    lose_trades.append(-1)  # Record a loss
                    break
                elif data['High'].iloc[j] >= take_profit:
                    win_trades.append(2)  # Record a win (2x risk)
                    break

    # Calculate win rate if there are trades for longs
    total_trades = len(win_trades) + len(lose_trades)
    if total_trades > 0:
        win_rate = len(win_trades) / total_trades * 100
        results['longs'][rsi_threshold] = win_rate
        print(f"RSI {rsi_threshold} Long: Win Rate = {win_rate:.2f}%")

# Now we handle the short side (RSI > 70)

short_rsi_levels = range(70, 90, 5)  # RSI levels for shorts (overbought)

for rsi_threshold in short_rsi_levels:
    print(f"Testing RSI short threshold: {rsi_threshold}")  # Debug print to track progress

    # Define a sell condition based on the RSI threshold (shorts)
    sell_condition = data['RSI'] > rsi_threshold

    # Track results for shorts
    win_trades = []
    lose_trades = []

    # Simulate trades for each sell signal
    for i in range(1, len(data)):
        if sell_condition.iloc[i]:  # Short condition
            entry_price = data['Close'].iloc[i]
            stop_loss = entry_price + data['ATR'].iloc[i]  # Risk 1x ATR as stop loss (above entry for shorts)
            take_profit = entry_price - 2 * (stop_loss - entry_price)  # 2x risk as take profit (below entry for shorts)

            # Simulate the trade outcome
            for j in range(i + 1, len(data)):
                if data['High'].iloc[j] >= stop_loss:
                    lose_trades.append(-1)  # Record a loss
                    break
                elif data['Low'].iloc[j] <= take_profit:
                    win_trades.append(2)  # Record a win (2x risk)
                    break

    # Calculate win rate if there are trades for shorts
    total_trades = len(win_trades) + len(lose_trades)
    if total_trades > 0:
        win_rate = len(win_trades) / total_trades * 100
        results['shorts'][rsi_threshold] = win_rate
        print(f"RSI {rsi_threshold} Short: Win Rate = {win_rate:.2f}%")

# Display results
print("\nFinal Results for Longs:")
for rsi_threshold, win_rate in results['longs'].items():
    if win_rate > 50:
        print(f"RSI Long Threshold: {rsi_threshold}, Win Rate: {win_rate:.2f}%")

print("\nFinal Results for Shorts:")
for rsi_threshold, win_rate in results['shorts'].items():
    if win_rate > 50:
        print(f"RSI Short Threshold: {rsi_threshold}, Win Rate: {win_rate:.2f}%")


[*********************100%***********************]  1 of 1 completed


Testing RSI long threshold: 20
RSI 20 Long: Win Rate = 54.55%
Testing RSI long threshold: 25
RSI 25 Long: Win Rate = 37.68%
Testing RSI long threshold: 30
RSI 30 Long: Win Rate = 30.66%
Testing RSI long threshold: 35
RSI 35 Long: Win Rate = 33.89%
Testing RSI long threshold: 40
RSI 40 Long: Win Rate = 35.10%
Testing RSI long threshold: 45
RSI 45 Long: Win Rate = 34.30%
Testing RSI long threshold: 50
RSI 50 Long: Win Rate = 33.98%
Testing RSI long threshold: 55
RSI 55 Long: Win Rate = 34.22%
Testing RSI long threshold: 60
RSI 60 Long: Win Rate = 34.05%
Testing RSI long threshold: 65
RSI 65 Long: Win Rate = 34.29%
Testing RSI short threshold: 70
RSI 70 Short: Win Rate = 30.36%
Testing RSI short threshold: 75
RSI 75 Short: Win Rate = 28.17%
Testing RSI short threshold: 80
RSI 80 Short: Win Rate = 15.38%
Testing RSI short threshold: 85
RSI 85 Short: Win Rate = 0.00%

Final Results for Longs:
RSI Long Threshold: 20, Win Rate: 54.55%

Final Results for Shorts:


In [ ]:
import yfinance as yf
import pandas as pd

# Load historical data (you can change 'AAPL' to any other symbol)
data = yf.download('EURUSD=X', start='2000-01-01', end='2023-01-01')

# Function to identify pivot highs (23 left, 1 right)
def is_pivot_high(data, left_candles=23, right_candles=1):
    data['Pivot_High'] = None
    highs = data['High']
    for i in range(left_candles, len(data) - right_candles):
        if all(highs.iloc[i] > highs.iloc[i - j] for j in range(1, left_candles + 1)) and \
           all(highs.iloc[i] > highs.iloc[i + j] for j in range(1, right_candles + 1)):
            data.loc[i, 'Pivot_High'] = highs.iloc[i]
    return data

# Function to identify pivot lows (23 left, 1 right)
def is_pivot_low(data, left_candles=23, right_candles=1):
    data['Pivot_Low'] = None
    lows = data['Low']
    for i in range(left_candles, len(data) - right_candles):
        if all(lows.iloc[i] < lows.iloc[i - j] for j in range(1, left_candles + 1)) and \
           all(lows.iloc[i] < lows.iloc[i + j] for j in range(1, right_candles + 1)):
            data.loc[i, 'Pivot_Low'] = lows.iloc[i]
    return data

# Apply pivot high/low detection for 23/1 configuration
data = is_pivot_high(data, 23, 1)
data = is_pivot_low(data, 23, 1)

# Initialize trading performance variables
initial_balance = 10000
balance = initial_balance
trade_risk = 0.01  # Risk 1% of balance per trade
risk_reward_ratio = 2  # 1:2 RR
trade_history = []

# Simulate trades based on pivot highs and lows
for i in range(len(data) - 1):
    if pd.notna(data.iloc[i]['Pivot_Low']) and i + 1 < len(data):
        # Buy signal at Pivot Low
        entry_price = data['Close'].iloc[i + 1]  # Enter on the next candle's close
        stop_loss = data['Low'].iloc[i] * (1 - 0.01)  # SL set at 1% below the pivot low
        take_profit = entry_price + (entry_price - stop_loss) * risk_reward_ratio  # 1:2 RR

        # Simulate trade
        for j in range(i + 2, len(data)):  # Start from the next day
            if data['Low'].iloc[j] <= stop_loss:
                loss = -balance * trade_risk
                balance += loss
                trade_history.append(('Buy', entry_price, stop_loss, data['Close'].iloc[j], 'Loss', loss))
                break
            elif data['High'].iloc[j] >= take_profit:
                profit = balance * trade_risk * risk_reward_ratio
                balance += profit
                trade_history.append(('Buy', entry_price, stop_loss, data['Close'].iloc[j], 'Win', profit))
                break

    elif pd.notna(data.iloc[i]['Pivot_High']) and i + 1 < len(data):
        # Sell signal at Pivot High
        entry_price = data['Close'].iloc[i + 1]  # Enter on the next candle's close
        stop_loss = data['High'].iloc[i] * (1 + 0.01)  # SL set at 1% above the pivot high
        take_profit = entry_price - (stop_loss - entry_price) * risk_reward_ratio  # 1:2 RR

        # Simulate trade
        for j in range(i + 2, len(data)):  # Start from the next day
            if data['High'].iloc[j] >= stop_loss:
                loss = -balance * trade_risk
                balance += loss
                trade_history.append(('Sell', entry_price, stop_loss, data['Close'].iloc[j], 'Loss', loss))
                break
            elif data['Low'].iloc[j] <= take_profit:
                profit = balance * trade_risk * risk_reward_ratio
                balance += profit
                trade_history.append(('Sell', entry_price, stop_loss, data['Close'].iloc[j], 'Win', profit))
                break

# Convert trade history to DataFrame for further analysis
trade_df = pd.DataFrame(trade_history, columns=['Type', 'Entry Price', 'Stop Loss', 'Exit Price', 'Result', 'P/L'])

# Calculate performance metrics
total_trades = len(trade_df)
win_trades = trade_df[trade_df['Result'] == 'Win']
lose_trades = trade_df[trade_df['Result'] == 'Loss']
win_rate = len(win_trades) / total_trades * 100 if total_trades > 0 else 0
net_profit = balance - initial_balance
max_drawdown = ((initial_balance - trade_df['P/L'].cumsum().min()) / initial_balance) * 100

# Display results
print(f'Total Trades: {total_trades}')
print(f'Win Rate: {win_rate:.2f}%')
print(f'Net Profit: ${net_profit:.2f}')
print(f'Max Drawdown: {max_drawdown:.2f}%')

# Show first few trades
trade_df.head()


[*********************100%***********************]  1 of 1 completed


Total Trades: 0
Win Rate: 0.00%
Net Profit: $0.00
Max Drawdown: nan%


,Type,Entry Price,Stop Loss,Exit Price,Result,P/L


In [ ]:
import yfinance as yf
import pandas as pd

# Load historical data
data = yf.download('EURUSD=X', start='2000-01-01', end='2023-01-01')

# Function to identify pivot highs
def is_pivot_high(data, left_candles, right_candles):
    highs = data['High']
    for i in range(left_candles, len(data) - right_candles):
        # Ensure we're accessing the data with iloc to avoid future warnings
        if all(highs.iloc[i] > highs.iloc[i - j] for j in range(1, left_candles + 1)) and \
           all(highs.iloc[i] > highs.iloc[i + j] for j in range(1, right_candles + 1)):
            data.loc[i, 'Pivot_High'] = highs.iloc[i]
    return data

# Function to identify pivot lows
def is_pivot_low(data, left_candles, right_candles):
    lows = data['Low']
    for i in range(left_candles, len(data) - right_candles):
        # Ensure we're accessing the data with iloc to avoid future warnings
        if all(lows.iloc[i] < lows.iloc[i - j] for j in range(1, left_candles + 1)) and \
           all(lows.iloc[i] < lows.iloc[i + j] for j in range(1, right_candles + 1)):
            data.loc[i, 'Pivot_Low'] = lows.iloc[i]
    return data

# Initialize results to store combinations
results = []

# Loop through combinations of left candles (starting from 5) and right candles (from 0 to 3)
for left_candles in range(5, 50):  # Left candles from 5 to 20
    for right_candles in range(0, 4):  # Right candles from 0 to 3
        # Reset columns for each loop
        data['Pivot_High'] = None
        data['Pivot_Low'] = None

        # Identify pivots based on the current combination of candles
        data = is_pivot_high(data, left_candles, right_candles)
        data = is_pivot_low(data, left_candles, right_candles)

        # Generate buy/sell signals based on pivot highs and lows
        buy_signals = data[data['Pivot_Low'].notnull()]
        sell_signals = data[data['Pivot_High'].notnull()]

        # Calculate metrics
        total_trades = len(buy_signals) + len(sell_signals)
        if total_trades > 0:
            win_rate = len(buy_signals) / total_trades * 100  # Example win rate (you can calculate based on real PnL)
            results.append({
                'Left Candles': left_candles,
                'Right Candles': right_candles,
                'Total Trades': total_trades,
                'Win Rate (%)': win_rate
            })

# Convert results to DataFrame and display
results_df = pd.DataFrame(results)
results_df.sort_values(by='Win Rate (%)', ascending=False, inplace=True)
results_df


[*********************100%***********************]  1 of 1 completed


,Left Candles,Right Candles,Total Trades,Win Rate (%)
179,49,3,222,53.603604
175,48,3,224,53.125000
119,34,3,279,53.046595
176,49,0,768,52.994792
171,47,3,227,52.863436
...,...,...,...,...
26,11,2,601,48.086522
41,15,1,651,48.079877
46,16,2,487,47.843943
30,12,2,576,47.743056
